# 🎓 向量数据库互动教程

这是一个交互式的向量数据库学习笔记本。你可以逐个运行单元格来学习概念。

**学习目标：**
- 理解嵌入模型的工作原理
- 掌握向量数据库的基本操作
- 构建简单的语义搜索应用

## 🔧 环境设置

In [ ]:
# 安装必要依赖
!pip install sentence-transformers faiss-cpu numpy matplotlib

In [ ]:
# 导入必要的库
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import time

print("✅ 环境设置完成！")

## 🧠 第一步：理解嵌入模型

In [ ]:
# 加载预训练的嵌入模型
print("🚀 加载嵌入模型...")
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print("✅ 模型加载完成！")

In [ ]:
# 看看文本是如何变成数字的
sample_text = "人工智能很有趣"
embedding = model.encode([sample_text])[0]

print(f"原文本: {sample_text}")
print(f"向量维度: {len(embedding)}")
print(f"前10个数值: {embedding[:10]}")
print(f"数值范围: {embedding.min():.3f} 到 {embedding.max():.3f}")

In [ ]:
# 比较不同文本的相似度
texts = [
    "我喜欢编程",
    "我热爱写代码", 
    "今天天气很好",
    "今日阳光明媚"
]

embeddings = model.encode(texts)

# 计算相似度矩阵
print("📝 文本相似度矩阵:")
print("\n文本列表:")
for i, text in enumerate(texts):
    print(f"{i+1}. {text}")

print("\n相似度分数 (越接近1越相似):")
print("     ", end="")
for i in range(len(texts)):
    print(f"  [{i+1}]", end="")
print()

for i in range(len(texts)):
    print(f"[{i+1}]  ", end="")
    for j in range(len(texts)):
        # 余弦相似度
        similarity = np.dot(embeddings[i], embeddings[j]) / \
                    (np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j]))
        print(f"{similarity:.2f} ", end="")
    print()

## 🗄️ 第二步：构建向量数据库

In [ ]:
# 创建示例数据集
knowledge_base = [
    "Python是一种解释型、面向对象的编程语言",
    "Java是一种跨平台的面向对象编程语言",
    "人工智能是计算机科学的一个重要分支领域",
    "机器学习是实现人工智能的核心技术之一",
    "深度学习是机器学习的一个重要子领域",
    "TensorFlow是由Google开发的机器学习框架",
    "PyTorch是Facebook开发的深度学习框架"
]

print("📚 知识库内容:")
for i, item in enumerate(knowledge_base):
    print(f"{i+1}. {item}")

In [ ]:
# 将知识库转换为向量
print("🔄 将文本转换为向量...")
start_time = time.time()
knowledge_vectors = model.encode(knowledge_base)
end_time = time.time()

print(f"✅ 转换完成！耗时: {end_time - start_time:.2f} 秒")
print(f"📊 向量形状: {knowledge_vectors.shape}")

In [ ]:
# 创建FAISS向量数据库
dimension = knowledge_vectors.shape[1]
index = faiss.IndexFlatL2(dimension)

print(f"🏗️ 创建 {dimension} 维的向量索引...")
index.add(knowledge_vectors)
print(f"✅ 向量数据库创建完成，包含 {index.ntotal} 条记录")

## 🔍 第三步：执行语义搜索

In [ ]:
# 定义搜索函数
def semantic_search(query, k=3):
    """执行语义搜索"""
    print(f"\n🔍 查询: \"{query}\"")
    
    # 将查询转换为向量
    query_vector = model.encode([query])
    
    # 执行搜索
    distances, indices = index.search(query_vector, k)
    
    print("\n🎯 搜索结果:")
    print("-" * 50)
    for i, (idx, dist) in enumerate(zip(indices[0], distances[0])):
        print(f"{i+1}. {knowledge_base[idx]}")
        print(f"   相似度分数: {dist:.2f}")
        print()
    
    return indices[0], distances[0]

In [ ]:
# 测试不同的查询
queries = [
    "什么是Python？",
    "AI技术介绍",
    "深度学习框架有哪些？"
]

for query in queries:
    semantic_search(query)

## 📊 第四步：可视化向量空间

In [ ]:
# 使用t-SNE降维进行可视化
print("🎨 降维处理中...")
tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(knowledge_base)-1))
vectors_2d = tsne.fit_transform(knowledge_vectors)

print("✅ 降维完成！")

In [ ]:
# 创建可视化图表
plt.figure(figsize=(12, 8))

# 绘制点
scatter = plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], 
                     c=range(len(knowledge_base)), 
                     cmap='viridis', 
                     alpha=0.7,
                     s=100)

# 添加标签
for i, (x, y) in enumerate(vectors_2d):
    plt.annotate(f'{i+1}', (x, y), xytext=(5, 5), 
                textcoords='offset points', fontsize=10,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

plt.title('知识库向量空间可视化 (t-SNE)', fontsize=16)
plt.xlabel('第一主成分')
plt.ylabel('第二主成分')
plt.grid(True, alpha=0.3)

# 添加图例
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                             markerfacecolor=plt.cm.viridis(i/len(knowledge_base)), 
                             markersize=10, label=f'{i+1}. {text[:30]}...') 
                   for i, text in enumerate(knowledge_base)]

plt.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()

## 🎯 第五步：构建简易问答系统

In [ ]:
# 创建问答系统类
class SimpleQASystem:
    def __init__(self, knowledge_base):
        self.knowledge_base = knowledge_base
        self.model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        self.vectors = self.model.encode(knowledge_base)
        
        # 创建向量数据库
        dimension = self.vectors.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(self.vectors)
        
        print("✅ 问答系统初始化完成！")
    
    def answer(self, question, k=1):
        """回答问题"""
        # 转换问题为向量
        question_vector = self.model.encode([question])
        
        # 搜索最相似的知识
        distances, indices = self.index.search(question_vector, k)
        
        # 返回答案
        answers = []
        for idx, dist in zip(indices[0], distances[0]):
            answers.append({
                'answer': self.knowledge_base[idx],
                'confidence': 1/(1 + dist),  # 简单的置信度计算
                'similarity_score': dist
            })
        
        return answers[0] if len(answers) == 1 else answers

In [ ]:
# 测试问答系统
qa_system = SimpleQASystem(knowledge_base)

test_questions = [
    "Python是什么？",
    "介绍一下AI",
    "有哪些深度学习框架？"
]

print("🤖 问答系统测试:")
print("=" * 50)

for question in test_questions:
    result = qa_system.answer(question)
    print(f"\n❓ 问: {question}")
    print(f"💡 答: {result['answer']}")
    print(f"📊 置信度: {result['confidence']:.3f}")
    print("-" * 30)

## 🚀 扩展练习

现在轮到你来尝试了！修改下面的代码：

In [ ]:
# TODO: 创建你自己的知识库
my_knowledge_base = [
    # 在这里添加你感兴趣的主题
    "在这里写下你的第一条知识...",
    "在这里写下你的第二条知识...",
    # 添加更多...
]

# TODO: 创建你自己的问答系统
# my_qa_system = SimpleQASystem(my_knowledge_base)

# TODO: 测试你的系统
# my_questions = ["你的问题1", "你的问题2"]
# for q in my_questions:
#     result = my_qa_system.answer(q)
#     print(f"问: {q}")
#     print(f"答: {result['answer']}")

## 📚 总结

恭喜完成本次学习！你现在已经掌握了：

✅ 嵌入模型的基本原理
✅ 向量数据库的创建和使用
✅ 语义搜索的实现
✅ 简易问答系统的构建

**下一步建议：**
1. 尝试不同的嵌入模型
2. 使用更大的数据集
3. 探索其他向量数据库（如 Weaviate、Milvus）
4. 添加更多的预处理和后处理功能